<a href="https://colab.research.google.com/github/toshiikuoo/puclic/blob/master/%E6%A0%AA%E4%BE%A1%E7%9B%B8%E9%96%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#必要なライブラリimport
import pandas as pd
from pandas import Series,DataFrame
from pandas_datareader import DataReader

import numpy as np

from datetime import datetime

import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('whitegrid')

from scipy.stats.stats import pearsonr
import itertools



In [0]:
#S&P全銘柄のリストを作成
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_list=pd.io.html.read_html(url)

In [13]:
sp500_list_first=sp500_list[0]
sp500_list_first

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,XYL,Xylem Inc.,reports,Industrials,Industrial Machinery,"White Plains, New York",2011-11-01,1524472,NaN
501,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,NaN
502,ZBH,Zimmer Biomet Holdings,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,NaN
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,NaN


In [14]:
#銘柄シンボルのみを取り出したSeriesを作成
sp500_symbol=sp500_list_first["Symbol"]
sp500_symbol

0       MMM
1       ABT
2      ABBV
3      ABMD
4       ACN
       ... 
500     XYL
501     YUM
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [15]:
#listに変換
sp500_symbol_list=sp500_symbol.values.tolist()
print(sp500_symbol_list)

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CDW', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CO

In [16]:
#yahooでは銘柄シンボルの"."は"-"に変えないとエラーになるので置換
#"."を含む要素を確認
sp500_symbol_list_in=[i for i in sp500_symbol_list if "." in i]
print(sp500_symbol_list_in)

['BRK.B', 'BF.B']


In [17]:
#"."を"-"に置換
sp500_symbol_list_yahoo=[i.replace(".","-")for i in sp500_symbol_list]
print(sp500_symbol_list_yahoo)


['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK-B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CDW', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CO

In [18]:
#確認のため"-"の含まれる要素を表示"
sp500_symbol_list_yahoo_in=[i for i in sp500_symbol_list_yahoo if "-" in i]
print(sp500_symbol_list_yahoo_in)

['BRK-B', 'BF-B']


In [19]:
#sp500銘柄の終値を１つのDataFrameに格納
close_sp500_list=DataReader(sp500_symbol_list_yahoo,'yahoo',datetime(2019, 9, 26, 7, 10, 37, 334315),datetime(2019, 10, 26, 7, 10, 37, 334315),)["Adj Close"]
close_sp500_list


Symbols,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AMG,AFL,A,APD,AKAM,ALK,ALB,ARE,ALXN,ALGN,ALLE,AGN,ADS,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,...,UHS,UNM,VFC,VLO,VAR,VTR,VRSN,VRSK,VZ,VRTX,VIAB,V,VNO,VMC,WAB,WMT,WBA,DIS,WM,WAT,WEC,WCG,WFC,WELL,WDC,WU,WRK,WY,WHR,WMB,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-26,163.830002,82.329941,71.905678,177.649994,191.298096,54.220001,283.809998,29.469999,160.380005,16.090879,82.970001,51.910000,76.355713,222.118286,90.580002,64.570000,67.190002,154.229996,98.699997,178.990005,102.660004,166.889999,127.779999,53.893238,108.110001,1242.290039,1241.390015,40.660000,1739.839966,9.61,80.379997,27.190001,94.290001,118.456947,55.880001,223.000000,125.129997,145.180664,82.000000,90.169998,...,146.559998,29.019449,88.070000,83.120003,116.730003,72.936142,191.190002,159.289993,59.817585,169.550003,24.049999,175.649994,63.049507,151.619995,71.930000,118.300003,54.080002,131.270004,115.089996,217.229996,95.900002,259.380005,48.869999,90.620003,61.256176,22.440001,36.330002,27.900000,154.839996,24.129999,197.370010,110.349998,65.570000,30.139999,97.239998,79.105408,112.809998,136.599991,43.959999,125.898628
2019-09-27,164.529999,81.513206,72.702156,175.889999,190.561264,52.330002,277.540009,28.719999,161.679993,16.180164,82.940002,51.889999,76.225998,219.899994,89.790001,64.180000,68.139999,153.110001,97.769997,175.839996,102.500000,168.250000,126.419998,53.664791,107.830002,1225.949951,1225.089966,40.130001,1725.449951,9.73,80.000000,27.010000,94.199997,118.138161,56.160000,218.979996,124.239998,146.591522,80.940002,91.370003,...,146.830002,29.306574,88.790001,84.440002,118.160004,72.857002,187.250000,155.589996,59.679024,166.740005,23.980000,174.000000,63.415787,149.039993,72.000000,118.449997,54.410000,129.960007,113.589996,222.529999,94.870003,259.290009,50.709999,90.290001,59.957706,23.049999,36.480000,27.790001,155.500000,24.200001,192.699997,108.250000,65.360001,30.330000,95.949997,79.903137,112.989998,134.130005,44.360001,123.741402
2019-09-30,164.399994,83.335899,73.547195,177.889999,191.527115,52.919998,276.250000,28.990000,165.399994,16.209925,83.349998,52.320000,76.629997,221.860001,91.379997,64.910004,69.519997,154.039993,97.940002,180.919998,103.650002,168.289993,128.130005,53.565468,108.680000,1221.140015,1219.000000,40.900002,1735.910034,9.75,80.050003,26.969999,93.690002,117.829346,55.700001,221.130005,124.230003,146.154373,82.330002,91.820000,...,148.750000,29.425385,88.989998,85.239998,119.089996,73.029999,188.630005,158.139999,59.738407,169.419998,24.030001,172.009995,63.029709,151.240005,71.860001,118.680000,55.310001,130.320007,115.000000,223.229996,95.099998,259.170013,50.439999,90.650002,59.115185,23.170000,36.450001,27.700001,158.360001,24.059999,192.970001,108.720001,64.889999,29.910000,95.900002,79.394585,113.430000,137.270004,44.520000,124.430511
2019-10-01,158.380005,81.513206,71.128639,170.210007,188.808792,52.900002,272.709991,28.760000,167.289993,16.110722,79.669998,51.540001,74.839996,216.479996,91.099998,64.949997,68.309998,153.139999,97.790001,182.300003,101.099998,166.460007,126.040001,53.525738,107.169998,1206.000000,1205.099976,41.160000,1735.650024,9.55,79.660004,26.799999,93.220001,116.255363,54.369999,220.889999,123.860001,139.835251,82.120003,88.800003,...,147.110001,27.930353,89.010002,85.220001,115.599998,72.559998,183.949997,156.449997,59.233658,167.580002,23.950001,174.289993,62.128860,149.059998,68.379997,117.849998,54.700001,129.550003,114.870003,224.199997,95.190002,258.510010,49.060001,89.459999,58.074425,23.219999,34.860001,27.250000,154.789993,23.820000,187.929993,109.919998,64.949997,29.469999,92.040001,76.612480,112.889999,133.610001,42.970001,124.959839
2019-10-02,155.649994,79.212433,70.060196,163.389999,184.059204,52.250000,266.660004,28.309999,162.960007,15.575020,76.220001,50.580002,73.349998,210.470001,89.870003,62

In [20]:
#株のシンボルのlistを作成
columns=close_sp500_list.columns.to_list()
columns

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'ATO',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BBT',
 'BDX',
 'BRK-B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF-B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CBS',
 'CDW',
 'CE',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME

In [21]:
#すべての株のペアの組み合わせを出力
print(list(itertools.combinations(columns,2)))

[('MMM', 'ABT'), ('MMM', 'ABBV'), ('MMM', 'ABMD'), ('MMM', 'ACN'), ('MMM', 'ATVI'), ('MMM', 'ADBE'), ('MMM', 'AMD'), ('MMM', 'AAP'), ('MMM', 'AES'), ('MMM', 'AMG'), ('MMM', 'AFL'), ('MMM', 'A'), ('MMM', 'APD'), ('MMM', 'AKAM'), ('MMM', 'ALK'), ('MMM', 'ALB'), ('MMM', 'ARE'), ('MMM', 'ALXN'), ('MMM', 'ALGN'), ('MMM', 'ALLE'), ('MMM', 'AGN'), ('MMM', 'ADS'), ('MMM', 'LNT'), ('MMM', 'ALL'), ('MMM', 'GOOGL'), ('MMM', 'GOOG'), ('MMM', 'MO'), ('MMM', 'AMZN'), ('MMM', 'AMCR'), ('MMM', 'AEE'), ('MMM', 'AAL'), ('MMM', 'AEP'), ('MMM', 'AXP'), ('MMM', 'AIG'), ('MMM', 'AMT'), ('MMM', 'AWK'), ('MMM', 'AMP'), ('MMM', 'ABC'), ('MMM', 'AME'), ('MMM', 'AMGN'), ('MMM', 'APH'), ('MMM', 'ADI'), ('MMM', 'ANSS'), ('MMM', 'ANTM'), ('MMM', 'AON'), ('MMM', 'AOS'), ('MMM', 'APA'), ('MMM', 'AIV'), ('MMM', 'AAPL'), ('MMM', 'AMAT'), ('MMM', 'APTV'), ('MMM', 'ADM'), ('MMM', 'ARNC'), ('MMM', 'ANET'), ('MMM', 'AJG'), ('MMM', 'AIZ'), ('MMM', 'ATO'), ('MMM', 'T'), ('MMM', 'ADSK'), ('MMM', 'ADP'), ('MMM', 'AZO'), ('MMM'

In [0]:
correlations={}

In [0]:
#列ごとのペアで相関を算出
for cola,colb in itertools.combinations(columns,2):
  nas=np.logical_or(np.isnan(close_sp500_list.loc[:,cola]),np.isnan(close_sp500_list.loc[:,colb]))
  correlations[cola + '__'+ colb]=pearsonr(close_sp500_list.loc[:,cola][~nas],close_sp500_list.loc[:,colb][~nas])

In [0]:
#出力された結果"correlations"はリスト形式なのでDataFrameに変換
result=DataFrame.from_dict(correlations,orient='index')
result.columns=['PCC','p-value']
print(result.sort_values('PCC'))